In [1]:
from qick import *

# the main program class
from qick.asm_v2 import AveragerProgramV2

# for defining sweeps
from qick.asm_v2 import QickSpan, QickSweep1D
from qick.asm_v2 import QickProgramV2, AcquireProgramV2

## tProc Assembler
from qick.tprocv2_assembler import Assembler, Logger

# jupyter setup boilerplate
%matplotlib inline
import matplotlib.pyplot as plt


In [2]:
import sys
import time
import numpy as np
soc = QickSoc('qick_111_rfbv2.bit')


In [3]:
np.set_printoptions(threshold=sys.maxsize)

wr_buff = []
Data_qty = 1000

asm = """
.ALIAS loop_ind   r0
.ALIAS loop_cnt   r1
.ALIAS mem_dt     r2
.ALIAS aux       r15


REG_WR loop_cnt imm #1 
REG_WR loop_ind imm #1000
TIME inc_ref #100
REPS:
     WAIT @50
     DMEM_WR [loop_cnt] op -op(tuser)
     TIME #200 inc_ref 
     
     REG_WR loop_cnt op -op(loop_cnt + #1) 
     REG_WR loop_ind op -op(loop_ind - #1) -uf
     JUMP REPS -if(NZ)
     JUMP HERE
     
"""

p_txt, p_bin  = Assembler.str_asm2bin(asm)

print('Loading Program Memory:')
soc.tproc.Load_PMEM(p_bin)

print('Loading Data Memory:')
for i in range(Data_qty):
    wr_buff.append( [0, 0,0,0,0,0,0,0] )
soc.tproc.load_mem(2, wr_buff, 0)
#rd_buff = soc.tproc.read_mem(2, 0, Data_qty)
#print(rd_buff)

print('Start Core')
soc.tproc.start()
time.sleep(1)
print('Stop Core')
soc.tproc.stop()

print('---------------------------------------------')
print("Reading Data Memory:")
rd_buff = soc.tproc.read_mem(2, 0, Data_qty)
#print(rd_buff)

print('MAX', np.max(rd_buff))

err_qty = 0
wtime = np.zeros(Data_qty, dtype=np.int32)
for ind in range(Data_qty):
    wtime[ind] = rd_buff[ind][0]
    if (wtime[ind] > 100):
        print(ind, 'Time:',wtime[ind])
        err_qty += 1
print(wtime)



Loading Program Memory:
Loading Data Memory:
Start Core
Stop Core
---------------------------------------------
Reading Data Memory:
MAX 142757876
248 Time: 114
774 Time: 142757876
872 Time: 114
[          0          50          48          57          55          53
          51          48          58          54          53          51
          49          58          56          54          51          49
          58          56          54          52          48          47
          57          54          52          50          48          57
          55          52          50          48          57          55
          53          51          48          58          55          53
          51          49          58          56          53          63
          49          58          56          54          52          49
          59         -13          54          52          50          47
          57          55          52          50          48          57
  